### 预测

使用数据：

MERGE_CSV_PATH = "/Users/zhouzhan/Documents/codes/python_code/liver_disease/output/汇总表.csv"

使用模型：
1. 先用 lasso 进行特征选择， alpha = 0.05
2. 使用随机森林做预测

对比结果：

未做特征选择： 0.7358490566037735

做特征选择：   0.7547169811320755

In [ ]:
import pandas as pd
import numpy as np
import sys
sys.path.append(r'/Users/zhouzhan/Documents/codes/python_code/liver_disease/')
import constants
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LassoCV, Lasso
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


PATH = constants.MERGE_CSV_PATH
TO_PATH = constants.SELECTION_MERGE_CSV_PATH


class Prediction_v2021428():

    def __init__(self):
        df = pd.read_csv(PATH)
        cols = df.columns.values.tolist()
        # cols.remove('Unnamed: 0')
        cols.remove('INHOSPTIAL_ID')
        cols.remove('ZHENGHOU1')
        self.X = df[cols]
        self.y = df['ZHENGHOU1']
        self.df = df


    def split(self,X,y):
        X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state=0)
        ss = StandardScaler()
        X_train = ss.fit_transform(X_train)
        X_test = ss.transform(X_test)
        return X_train,X_test,y_train,y_test

    def lasso(self):
        X_train,X_test,y_train,y_test = self.split(self.X, self.y)

        model = Lasso(alpha=0.05, fit_intercept=False)
        model.fit(X_train, y_train)

        print(model.coef_.ravel())

        return model


    def lassoCV(self):
        X_train,X_test,y_train,y_test = self.split(self.X, self.y)
        
        model = LassoCV(alphas=[1.0,0.05,0.001], fit_intercept=False)
        model.fit(X_train, y_train)

        print(model.coef_)
        return model

    # 选择特征
    def selection(self, model):
        count = 0
        for index, row in self.df.iteritems():
            if index == 'INHOSPTIAL_ID' or index == 'ZHENGHOU1':
                continue
            if model.coef_.ravel()[count] > 0.00001 or model.coef_.ravel()[count] < -0.00001:
                count += 1
                print(index)
                continue
            self.df.drop(index,axis=1,inplace=True)
            count += 1

    # 输出 CSV 文件
    def toCSV(self,path):
        self.df.to_csv(path)

if __name__  == '__main__':
    pre = Prediction_v2021428()
    model = pre.lasso()
    
    pre.selection(model)
    pre.toCSV(TO_PATH)
    

### 对比

#### 特征选择前

特征：
>  ['身目尿黄', '腹胀', '发热', '纳差', '双下肢水肿', '乏力', '黑便', '头晕', '腹痛', '身目黄染', '恶心', '腹泻', '呕吐', '鼻塞', '肛门坠胀感', '大便次数增多', '皮肤瘙痒', '咳嗽', '咳痰', '呕血', '肝功能受损', '厌油', '全身胀痛', '陶土样大便', '精神差', '体重较前下降kg', '口苦', '大便干', '尿少', '行走困难', '易疲劳', '大便时干时稀', '夜寐差', '欲呕', '口干', '计算力下降', '里急后重', '水样便', '低热', '大便偏干', '气促', '大汗', '定向力迟钝', '计算力', '吐字不清', '扑翼样震颤阳性', '神志欠清', '四肢欠温', '尿急', '尿频', '头晕头痛', '反应迟钝', '无皮肤瘙痒', '尿痛', '反酸', '嗜睡', '咽部隐痛不适', '干呕', '鼻衄', '排便不爽', '口淡无味', '大便稀少', '大便未解', '脐疝', '不能言语交流', '双下肢无水肿', '肝区疼痛', '大便稀溏']

LogisticRegression()

0.7547169811320755

SVC(kernel='linear', random_state=0)

0.7547169811320755

DecisionTreeClassifier(max_depth=5)

0.7547169811320755

RandomForestClassifier(criterion='entropy')

0.7169811320754716

AdaBoostClassifier(n_estimators=100)

0.49056603773584906


#### 特征选择后

使用 lasso 选择出以下特征；
> 身目尿黄、双下肢水肿、乏力、黑便、腹痛、咳痰、全身胀痛、口苦、行走困难、欲呕、肝区疼痛


LogisticRegression()

0.7547169811320755

SVC(kernel='linear', random_state=0)

0.7924528301886793

DecisionTreeClassifier(max_depth=5)

0.7547169811320755

RandomForestClassifier(criterion='entropy')

0.7547169811320755

AdaBoostClassifier(n_estimators=100)

0.7358490566037735

#### 人工选择

主症：
> 右胁胀痛、身目黄染、恶心厌油、小便黄、腹胀、呕吐、厌油、身目尿黄、恶心、尿黄、胃痛、皮肤巩膜黄染、乏力

伴随：
> 尿少、腹胀、头晕、肝功能异常、心悸、纳差、乏力、黑便、目黄、厌油、发热、身目尿黄、胸闷、恶心、身目发黄、尿黄、神志欠清、身目黄染、双下肢水肿


LogisticRegression()

0.7619047619047619

SVC(kernel='linear', random_state=0)

0.7619047619047619

DecisionTreeClassifier(max_depth=5)

0.7619047619047619

RandomForestClassifier(criterion='entropy')

0.7857142857142857

AdaBoostClassifier(n_estimators=100)

0.7619047619047619